In [84]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [85]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1758489894,2025-09-21 18:24,1758489300,2025-09-21 18:15,19.1,66.4,0,Chuva fraca,//cdn.weatherapi.com/weather/64x64/night/296.png,1183,5.8,9.4,77,ENE,1009.0,29.8,0.83,0.03,94,75,19.1,66.4,19.7,67.4,19.7,67.4,18.1,64.6,10.0,6.0,0.0,8.7,14.0,"[{'date': '2025-09-21', 'date_epoch': 17584128..."


In [87]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-21,1758412800,"[{'time_epoch': 1758423600, 'time': '2025-09-2...",31.2,88.2,17.7,63.9,22.1,71.9,12.3,19.8,2.72,0.11,0.0,9.4,5.0,63,1,95,0,0,Possibilidade de chuva irregular,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,1.7,06:07 AM,06:13 PM,05:55 AM,06:11 PM,New Moon,1,0,0


In [88]:
df_hour = pd.json_normalize(hour_list)

# Converter para datetime (se ainda não estiver)
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())

df_5h

                 time    time_epoch  temp_c  temp_f  is_day  wind_mph  \
0 2025-09-21 00:00:00  1.758431e+09   18.08   64.58     0.0     4.120   
1 2025-09-21 05:00:00  1.758449e+09   22.54   72.58     0.6     8.540   
2 2025-09-21 10:00:00  1.758467e+09   29.50   85.10     1.0    10.780   
3 2025-09-21 15:00:00  1.758485e+09   21.60   70.90     0.8     4.940   
4 2025-09-21 20:00:00  1.758501e+09   18.00   64.45     0.0     6.425   

   wind_kph  wind_degree  pressure_mb  pressure_in  precip_mm  precip_in  \
0      6.62        83.40      1009.60      29.8140     0.0000     0.0000   
1     13.74       334.60      1008.40      29.7840     0.0000     0.0000   
2     17.34       305.00      1005.40      29.6860     0.0160     0.0000   
3      7.92       140.40      1006.80      29.7340     0.4960     0.0180   
4     10.35        40.25      1006.75      29.7325     0.0375     0.0025   

   snow_cm  humidity  cloud  feelslike_c  feelslike_f  windchill_c  \
0      0.0      65.4  18.20       

,time,time_epoch,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.code
0,2025-09-21 00:00:00,1.758431e+09,18.08,64.58,0.0,4.120,6.62,83.40,1009.60,29.8140,0.0000,0.0000,0.0,65.4,18.20,18.08,64.58,18.08,64.58,18.08,64.58,11.48,52.64,0.00,0.0,0.0,0.0,10.0,6.00,8.66,13.92,0.00,1000.60
1,2025-09-21 05:00:00,1.758449e+09,22.54,72.58,0.6,8.540,13.74,334.60,1008.40,29.7840,0.0000,0.0000,0.0,50.6,8.80,23.00,73.40,22.54,72.58,23.00,73.40,11.52,52.76,0.00,0.0,0.0,0.0,10.0,6.00,12.14,19.48,0.82,1000.00
2,2025-09-21 10:00:00,1.758467e+09,29.50,85.10,1.0,10.780,17.34,305.00,1005.40,29.6860,0.0160,0.0000,0.0,35.6,39.40,29.00,84.22,29.50,85.10,29.00,84.22,12.62,54.70,0.20,20.0,0.0,0.0,10.0,6.00,13.22,21.28,6.66,1014.40
3,2025-09-21 15:00:00,1.758485e+09,21.60,70.90,0.8,4.940,7.92,140.40,1006.80,29.7340,0.4960,0.0180,0.0,78.8,75.40,21.70,71.06,21.74,71.14,22.30,72.14,16.94,62.50,1.00,96.6,0.0,0.0,8.8,5.20,7.18,11.54,0.74,1158.40
4,2025-09-21 20:00:00,1.758501e+09,18.00,64.45,0.0,6.425,10.35,40.25,1006.75,29.7325,0.0375,0.0025,0.0,93.5,43.75,18.00,64.45,18.00,64.45,18.00,64.45,16.95,62.50,0.25,25.0,0.0,0.0,8.0,4.75,11.35,18.25,0.00,1068.25


In [89]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
0,2025-09-21 00:00:00,18.08,65.4,18.20,0.00,6.62,0.00,0.0
1,2025-09-21 05:00:00,23.00,50.6,8.80,0.82,13.74,0.00,0.0
4,2025-09-21 20:00:00,18.00,93.5,43.75,0.00,10.35,0.25,25.0
2,2025-09-21 10:00:00,29.00,35.6,39.40,6.66,17.34,0.20,20.0
3,2025-09-21 15:00:00,21.70,78.8,75.40,0.74,7.92,1.00,96.6


In [90]:
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
})

df_tabela1

,estilos,data e hora,temperatura,chance de chuva,umidade,uv
0,casual,2025-09-21 00:00:00,18.08,0.0,65.4,0.00
1,esportivo,2025-09-21 05:00:00,23.00,0.0,50.6,0.82
2,social,2025-09-21 10:00:00,29.00,20.0,35.6,6.66
3,streetwear,2025-09-21 15:00:00,21.70,96.6,78.8,0.74


In [91]:
# Garante que 'data e hora' seja do tipo datetime
df_tabela1['data e hora'] = pd.to_datetime(df_tabela1['data e hora'])

# Extrai hora, minuto, dia, etc.
df_tabela1['hora'] = df_tabela1['data e hora'].dt.hour
df_tabela1['dia'] = df_tabela1['data e hora'].dt.day
df_tabela1['mes'] = df_tabela1['data e hora'].dt.month

In [92]:
print(df_tabela1.columns.tolist())

['estilos', 'data e hora', 'temperatura', 'chance de chuva', 'umidade', 'uv', 'hora', 'dia', 'mes']


In [93]:
#transformando variáveis e em um número categorico 

X = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]  # todas numéricas
y = df_tabela1['estilos']  # variável alvo

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [95]:
# Supondo que df_tabela1 já tenha a coluna 'hora' extraída
X_dummies = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]

In [96]:
df_tabela1['predicao_estilo'] = clf.predict(X_dummies)
print(df_tabela1)


      estilos         data e hora  temperatura  chance de chuva  umidade  \
0      casual 2025-09-21 00:00:00        18.08              0.0     65.4   
1   esportivo 2025-09-21 05:00:00        23.00              0.0     50.6   
2      social 2025-09-21 10:00:00        29.00             20.0     35.6   
3  streetwear 2025-09-21 15:00:00        21.70             96.6     78.8   

     uv  hora  dia  mes predicao_estilo  
0  0.00     0   21    9          casual  
1  0.82     5   21    9          social  
2  6.66    10   21    9          social  
3  0.74    15   21    9      streetwear  
